# Assigment 4

In [1]:
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt

In [ ]:
def myQ(V, F, TT, hard_id, hard_value):
    assert(hard_id[0] > 0)
    assert(hard_id.shape[0] == hard_value.shape[0])
    T1, T2, _ = igl.local_basis(V, F)
    # Arrays for the entries of the matrix Q
    data = []
    ii = []
    jj = []
    Q = Q = sp.coo_matrix((data, (ii, jj)), shape=(F.shape[0], F.shape[0])).asformat("lil")
    b = np.zeros(F.shape[0], dtype=complex)
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            # Look up the opposite face
            g = TT[f, ei]
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue  
            # Compute the shared edge
            e = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            # ef, eg are the shared edge vector expressed in each local basis of f and g
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j)
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j)
            # System Assembly - smoothness
            Q[f, f] = Q[f, f] + ef * ef.conjugate()
            Q[f, g] = Q[f, g] - ef * eg.conjugate()
            Q[g, f] = Q[g, f] - eg * ef.conjugate()
            Q[g, g] = Q[g, g] + eg * eg.conjugate()
    for ci in range(hard_id.shape[0]):
        f = hard_id[ci]
        v = hard_value[ci, :]
        # Project onto the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        for i in range(F.shape[0]):
            b[i] = b[i] - Q[i,f] * c
            Q[i,f] = 0
    Q = Q.asformat("csr")
    u = sp.linalg.spsolve(Q, b)
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]
    return R

# Q1. Tangent vector fields for scalar field design

Specifies the field vectors at a subset of the mesh triangles (the constraints) and those constraints are interpolated smoothly throughout the surface to define a field.

## * Creating vector constraints - Done

## Interpolating the constraints

In [2]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
# tt: (#F x #3), i,j is the id of the triangle adj to the j edge of triangle i
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

In [3]:
# find root of a Complex Polynomial {c0}+{c1=1}*t+{c2=1}*t^2+...+{cn-1=1}*t^{n-1}+t^n
def find_root(c0,n):
    # M: companion matrix
    M = np.zeros((n,n),dtype=complex)
    for i in range(1,n):
        M[i,i-1] = 1.0
    M[0,n-1] = c0
    w, v = np.linalg.eig(M)
    return w[0]

In [4]:
# this given function implements the N-Directional Field
def align_field(V, F, TT, soft_id, soft_value, llambda, n):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])
    assert(n>0)
    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
       
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            ef = np.power(ef,n)
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            eg = np.power(eg,n)
            
            # System Assembly - N-PolyVectors
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    
    sqrtl = sqrt(llambda)
    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    
    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=complex)
    
    # System Assembly - Soft Constraints
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        c = np.power(c,n)
        
        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c * sqrtl
        
        index += 1
    
    assert(b.shape[0] == index)
    
    # Solve the linear system A^*Ax = A^*b
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.H @ A, A.H @ b)
    
    R = np.zeros_like(T1)
    
    for j in range(u.shape[0]):
        u[j] = find_root(u[j],n)
    
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

In [5]:
def plot_mesh_field(V, F, R, constrain_faces,n):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]

    # Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    
    # Convert R to an angle
    angles = np.arctan2(np.sum(R*T2,axis=1),np.sum(R*T1,axis=1))
    
    for i in range(n):
        R2 = np.cos(angles+i*2*math.pi/n)[:,None] * T1 + np.sin(angles+i*2*math.pi/n)[:,None] * T2
        p.add_lines(B, B + R2 * avg)
    
    return p

In [10]:
R = align_field(v, f, tt, cf, c, 1e6, 4)
p = plot_mesh_field(v, f, R, cf, 4)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Q2: Reconstructing a scalar field from a vector field

$$
\min s^TKs + s^Tb + c = \min A |g - u|^2 = \min A g^2 + A u^2 - 2 A g u
$$

equivalent to solving $s$ from

$$
2Ks + b = 0
$$

In [121]:

def reconstruct_scalar(V,F):
    # 2*Area (#F x 3)
    d_area = igl.doublearea(V, F)
    # gradient of phi of each triag t (3#F x #V)
    G = igl.grad(V, F, uniform = True)

    # s (#V x 1) vertex values
    data = []
    ii = []
    jj = []
    # for f in range(F.shape[0]):
    #     for ei in range(3): # Loop over the edges

print(igl.doublearea(v, f).shape)
G = igl.grad(v, f)
print(G.shape)

print(v.shape)
print(f.shape)

(756,)
(2268, 411)
(411, 3)
(756, 3)


# Q3: Harmonic and LSCM Parameterizations


In [68]:
# you will first map the mesh boundary to a unit circle in the UV plane centered at the origin.
# The boundary U and V coordinates are then "harmonically interpolated" 
# into the interior by solving the Laplace equation with Dirichlet boundary conditions 
# (setting the Laplacian of U equal to zero at each interior vertex, 
# then doing the same for V). 
#vThis involves two separate linear system solves (each with the same system matrix).



# harmonic weights
# igl.harmonic(v, f, b, bc, k)

def func(V, F):
    # compute largest ordered boundary loops in the given mesh, 
    # F (#F x 3) => bnd (#b x 1)
    bnd = igl.boundary_loop(F)
    # Map the vertices, 2D positions on the unit circle for the vertices in bnd, 
    # F (#F x dim=3), bnd (#b x 1) => bnd_uv (#b x 2) 2D pos on the unit circle for bnd vertices
    bnd_uv = map_vertices_to_circle(V, bnd)
    harmonic_w = igl.harmonic(V,F,bnd,bc,k=1)

    
    # Least-squares conformal map, 
    # bnd (#b x 1) boundary indices, bc (#b x 2) list of boundary values
    uv_lscm = igl.lscm(V,F,bnd,bc)

    
# p = mp.subplot(v, f, uv=uv, s=[1, 2, 0])
# mp.subplot(uv, f, uv=uv, shading={"wireframe": True}, data=p, s=[1, 2, 1])

array([  0, 358,  58, 187,  18, 183,  57, 180,   5, 198,  52, 203,  15,
       207,  43, 210,   1, 211,  51, 135,  16, 132,  37, 124,   7, 121,
        33, 115,  13, 116,  28, 229,   2, 230,  25, 218,  24, 217,  26,
       224,   8, 222,  79, 247,  11, 273,  83, 275,   3, 392,  84, 279,
        20, 259,  70, 319,   6, 316,  62, 296,  19, 292,  59, 359])

# 4. Editing a parameterization with vector fields